In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
root = '/mnt/c/Users/tsimk/Downloads/dotfiles'
msk_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/camacity/dat/cty_msk_/city_clrd0000.gl5'
msk = np.fromfile(msk_path, dtype='float32').reshape(2160, 4320)

In [ ]:
city_num = 1
tmp = np.where(msk==1, 1, 0)